### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D16249B10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D16754450>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]
result=model.invoke(messages)

In [4]:
result

AIMessage(content='Here are a couple of ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to greet someone)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual way to greet someone you know well)\n\n\n\nLet me know if you\'d like to see other variations!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 21, 'total_tokens': 111, 'completion_time': 0.163636364, 'prompt_time': 0.00021213, 'queue_time': 0.013064290000000001, 'total_time': 0.163848494}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d0750a7-b99c-4459-8003-1d15768847bd-0', usage_metadata={'input_tokens': 21, 'output_tokens': 90, 'total_tokens': 111})

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a couple of ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to greet someone)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual way to greet someone you know well)\n\n\n\nLet me know if you\'d like to see other variations!\n'

In [6]:
### Using LCEL - chain the components
chain=model|parser
chain.invoke(messages)

"Hello: Bonjour \n\nHow are you?: Comment allez-vous? (formal) or Comment vas-tu? (informal) \n\n\nLet me know if you'd like me to translate any other phrases!\n"

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# Generic template definition
generic_template = "Translate the following into {language}:"

# Creating a chat prompt template correctly
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

print(prompt)


input_variables=['language', 'text'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})]


In [8]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [9]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [10]:
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [11]:
print("The End")

The End
